# Atualiando o Plotly
!pip install -U plotly

In [ ]:
# Atualiando o Plotly
!pip install -U plotly

In [1]:
!pip install -q yfinance

     |████████████████████████████████| 6.3MB 8.1MB/s 


In [1]:
import yfinance as yf
from datetime import date
import plotly.express as px

import pandas as pd
pd.options.plotting.backend = "plotly"

In [2]:
def consulta_bc(codigo_serie, dataInicial='01/01/1900',  dataFinal=date.today().strftime('%d/%m/%Y')):  
  url = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={dataInicial}&dataFinal={dataFinal}'
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  return df

In [3]:
data_inicial = '01/01/2000'
data_final   = '25/03/2021'

In [5]:
ibov = yf.download('^BVSP', start=pd.to_datetime(data_inicial, dayfirst=True), end=pd.to_datetime(data_final, dayfirst=True))['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [6]:
ibov

Date
2000-01-03     16930.0
2000-01-04     15851.0
2000-01-05     16245.0
2000-01-06     16107.0
2000-01-07     16309.0
                ...   
2021-03-18    114835.0
2021-03-19    116222.0
2021-03-22    114979.0
2021-03-23    113262.0
2021-03-24    112064.0
Name: Adj Close, Length: 5248, dtype: float64

## CDI

In [7]:
cdi = consulta_bc(12, data_inicial, data_final)

In [9]:
cdi.columns = ['diaria']

In [ ]:
cdi

In [12]:
cdi['acumulado'] = (1 + (cdi['diaria'] / 100)).cumprod().shift(1)
cdi['acumulado'].iloc[0] = 1

In [ ]:
cdi

In [14]:
df_ibov_cdi = pd.concat([ibov / ibov.iloc[0], cdi['acumulado']], axis=1).dropna()

In [15]:
df_ibov_cdi.columns = ['IBOV', 'CDI']

In [16]:
df_ibov_cdi

,IBOV,CDI
2000-01-03,1.000000,1.000685
2000-01-04,0.936267,1.001368
2000-01-05,0.959539,1.002052
2000-01-06,0.951388,1.002735
2000-01-07,0.963320,1.003419
...,...,...
2021-03-18,6.782930,11.645860
2021-03-19,6.864855,11.647069
2021-03-22,6.791435,11.648278
2021-03-23,6.690018,11.649487


In [17]:
(df_ibov_cdi *100).plot()